# <span style="color:#ff5f27"> 📝 Imports

In [1]:
import pandas as pd
from datetime import datetime, date

import requests
import json

# <span style="color:#ff5f27"> 🏙️ Cities coordinates

In [2]:
city_coordinates = {
    'Kyiv': [50.5, 30.5],  # latitude, longitude
    'London': [51.5, -0.099990845],
    'Paris': [48.90001, 2.4000092],
    'Stockholm': [59.300003, 18.100006],
    'New_York': [40.699997, -74],
    'Los_Angeles': [34.1, -118.2],
    'Singapore': [1.4000015, 103.80002],
    'Sidney': [40.300003, 84.2],
    'Hong_Kong': [22.300003, 114.20001],
    'Rome': [41.90001, 12.5]
}

# <span style="color:#ff5f27"> 🔮 Data Parsing

In [3]:
def parse_weather_data(city_name, today, coordinates=city_coordinates):
    
    latitude, longitude = coordinates[city_name]
    
    url = f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={today}&end_date={today}&hourly=temperature_2m,relativehumidity_2m,precipitation,weathercode,windspeed_10m,winddirection_10m'
    response = requests.get(url)

    assert response.status_code == 200, f"⚠️ Response status code is: {response.status_code}"
    
    data_json = json.loads(response.content)['hourly']

    date = data_json['time']
    temperature = data_json['temperature_2m']
    humidity = data_json['relativehumidity_2m']
    precipitation = data_json['precipitation']
    weather_condition = data_json['weathercode']
    wind_speed = data_json['windspeed_10m']
    wind_direction = data_json['winddirection_10m'] 
    
    data_frame = pd.DataFrame(
        {
            'date': date,
            'weather_condition': weather_condition,
            'temperature': temperature,
            'humidity': humidity,
            'precipitation': precipitation,
            'wind_speed': wind_speed,
            'wind_direction': wind_direction
        }
    )
    data_frame.date = data_frame.date.str.replace('T',' ')
    data_frame.date = pd.to_datetime(data_frame.date, format='%Y-%m-%d %H:%M')
    data_frame['city_name'] = city_name
    
    return data_frame[
        [
            'city_name',
            'date',
            'weather_condition',
            'temperature',
            'humidity',
            'precipitation',
            'wind_speed',
            'wind_direction'
        ]
    ]

In [4]:
#today = date.today()
today = '2023-01-31'

data_parsed = pd.DataFrame()

for city in city_coordinates.keys():
    parsed_df = parse_weather_data(
        city,
        today
    )
    data_parsed = pd.concat([data_parsed,parsed_df])
    
data_parsed

,city_name,date,weather_condition,temperature,humidity,precipitation,wind_speed,wind_direction
0,Kyiv,2023-01-31 00:00:00,73,-0.7,87,0.3,14.3,225
1,Kyiv,2023-01-31 01:00:00,73,-0.8,88,0.3,13.3,229
2,Kyiv,2023-01-31 02:00:00,71,-0.8,88,0.1,12.3,232
3,Kyiv,2023-01-31 03:00:00,71,-0.9,88,0.1,11.3,239
4,Kyiv,2023-01-31 04:00:00,3,-0.8,88,0.0,10.6,246
...,...,...,...,...,...,...,...,...
19,Rome,2023-01-31 19:00:00,0,9.6,64,0.0,0.7,360
20,Rome,2023-01-31 20:00:00,0,9.4,65,0.0,1.5,315
21,Rome,2023-01-31 21:00:00,0,8.5,67,0.0,3.1,54
22,Rome,2023-01-31 22:00:00,0,6.3,74,0.0,4.2,59


---
# <span style="color:#ff5f27"> 🔮 Connecting to Hopsworks Feature Store

In [8]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

weather_historical_fg = fs.get_or_create_feature_group(
    name='weather_historical_fg',
    version=1
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3346
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27"> ⬆️ Uploading new data to the Feature Store

In [9]:
weather_historical_fg.insert(data_parsed)

Uploading Dataframe: 0.00% |          | Rows 0/240 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3346/jobs/named/weather_historical_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fea0378a190>, None)

---